In [1]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date
hoje =  pd.to_datetime(date.today())

Current working directory was changed to: C:\Users\viniciusbarros\Documents\GitHub\Previsor\App


In [2]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 10/01/23 - Horário: 13:25:50.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 10/01/23 - Horário: 13:28:47.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 10/01/23 - Horário: 13:28:52.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 10/01/23 - Horário: 13:29:03.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 10/01/23 - Horário: 13:30:19.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 10/01/23 - Horário: 13:31:28.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 10/01/23 - Horário: 13:31:40.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 10/01/23 - Horário: 13:32:20.  Portanto não foi baixado novamente.
vrapeelo

In [3]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
ide_data_cols = ['DthEnvio','IdeUsinaOutorga']
ide_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Baixa bancos de dados

In [4]:
# Carrega BD vmonitoramentoleilao => monitoramentoleilao
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao.sample(5)

,IdeUsinaOutorga,CodLeilao,DatInicioSuprimento
2620,51590,06/2021,2024-01-01
3035,55998,01/2021,2022-05-01
2283,49691,06/2021,2024-01-01
163,31652,10/2013,2018-05-01
1024,38035,03/2018,2024-01-01


In [5]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DthEnvio' : 'Data_Envio',
        'NomUsina' : 'usina_nome',
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma.sample(5)

,IdeUsinaOutorga,Data_Envio,canteiroReal,prev_IO,IO_real,DesvRio_real,Ench_Real,IOTrans_Real,Conc_Trans_REAL,Comiss_Real,ME_Real_conc_eol,Conc_Combust_Real,DatRealizacaoVI,DatRealizacaoX,DatRealizacaoV,DatRealizacaoIV
34087,54715,2022-10-04 23:17:12.697,NaT,2024-02-15,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
26546,35828,2022-06-03 12:24:07.380,2021-08-06,NaT,2021-08-09,NaT,NaT,2021-11-22,2022-01-17,2022-01-13,NaT,2022-02-21,NaT,2021-10-20,NaT,NaT
17207,44566,2021-11-04 15:42:43.377,NaT,2022-04-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
18799,49728,2021-12-03 18:29:57.293,NaT,2025-01-20,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
31770,51531,2022-09-02 15:47:32.223,2022-08-01,2022-12-16,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [6]:
# Carrega BD vrapeelcontratorecurso => mercado

cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)

# Carrega BD vrapeellicenciamento => ambiental_datas
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeelacesso => acesso_contratos
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]

# Cria dataframe input_rapeel

In [7]:
# Carrega BD vrapeelacesso => acesso_contratos
input_rapeel = pd.merge(mercado,ambiental_datas,on=ide_data_cols,how="inner")
input_rapeel = pd.merge(input_rapeel,acesso_contratos,on=ide_data_cols,how="inner")
input_rapeel = input_rapeel.loc[input_rapeel.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
input_rapeel.sample(5)

,DthEnvio,IdeUsinaOutorga,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT
1432,2023-01-03 10:29:11.757,54778,NaT,NaT,2024-11-19,NaT,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado
1923,2023-01-04 11:54:27.650,50548,NaT,NaT,2024-02-15,NaT,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado
2443,2023-01-05 17:01:52.087,47052,NaT,2030-03-08,2030-03-08,2030-03-08,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado
131,2021-03-04 15:30:33.393,34634,2020-02-06,2022-05-05,2019-04-04,2026-02-22,Não se Aplica,Válido,Não se Aplica,Válido
1216,2023-01-02 13:57:41.703,55344,NaT,NaT,2022-11-19,NaT,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado


In [8]:
definicoes =   input_rapeel #pd.merge(input_rapeel,ug_rapeel,on=ide_data_cols,how='inner')
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]
definicoes[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
638,52082,2031-10-08,2031-10-08,2031-10-08,2031-10-08
634,49815,2031-03-05,2031-03-05,2031-03-05,2031-03-05
72,33533,2025-08-10,2023-02-27,2021-10-20,2025-08-10
2505,46889,NaT,2026-07-31,2025-01-15,2026-07-31
1622,33415,NaT,2027-04-03,2025-12-08,2027-04-03


In [9]:
cols = ["IdeUsinaOutorga","DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]

monitoramentodados = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]

In [10]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao
rename_cols = {'DatPrevistaComercial':'DatPrevistaComercial'}
definicoes_monitoramento = pd.merge(monitoramentodados,definicoes,on='IdeUsinaOutorga',how="left").rename(columns=rename_cols).drop_duplicates()
definicoes_monitoramento = definicoes_monitoramento.loc[definicoes_monitoramento.IdcUsinaMonitorada == "Sim"]

definicoes_monitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoes_monitoramento.loc[definicoes_monitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()) & (definicoes_monitoramento.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


definicoes_monitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(definicoes_monitoramento[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

definicoes_monitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSitContratoCCD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCCT == nsa) &
    definicoes_monitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoes_monitoramento.loc[
(    ((definicoes_monitoramento.NomSitContratoCCD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCCT == valido)) &
    definicoes_monitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSitContratoCCD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCCT == vencido) |
    (definicoes_monitoramento.NomSitContratoCCD == nass) |
    (definicoes_monitoramento.NomSitContratoCCT == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoes_monitoramento['CondicaoUso'] = "Não informado"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCUST == vencido) |
    (definicoes_monitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoes_monitoramento.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoes_monitoramento["PPA"] = "Ambos"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2020-12-11,2017-09-18,LI,2020-12-11
1,NaT,2026-03-09,2002-12-11,LI,2026-03-09
2,NaT,NaT,NaT,Sem LP,NaT
3,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
7,NaT,NaT,NaT,Não Informado,NaT
...,...,...,...,...,...
3093,NaT,NaT,NaT,Não Informado,NaT
3094,NaT,NaT,NaT,Não Informado,NaT
3095,NaT,NaT,NaT,Não Informado,NaT
3096,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
2318,49856,Não se Aplica,Não Assinado,2023-01-02 12:19:24.237,Sem Conexão
2162,49469,Não se Aplica,Válido,2023-01-03 00:39:54.677,OK
1879,47452,Não se Aplica,Não Assinado,2023-01-02 20:14:07.007,Sem Conexão
2349,49926,Não se Aplica,Não Assinado,2023-01-05 12:25:37.700,Sem Conexão
157,31562,Não se Aplica,Não Assinado,2023-01-05 15:08:26.427,Sem Conexão
501,34778,Não se Aplica,Não Assinado,2023-01-04 17:45:10.807,Sem Conexão
2431,50210,Não se Aplica,Não Assinado,2023-01-04 17:44:30.200,Sem Conexão
1636,46781,Não se Aplica,Válido,2023-01-04 11:56:27.260,OK
1428,44575,Não se Aplica,Válido,2023-01-04 18:51:33.423,OK
359,33807,NaN,NaN,NaT,Não informado


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
1026,38078,Não se Aplica,Válido,OK
1615,46702,Não se Aplica,Não se Aplica,Verificar
1056,38130,Não se Aplica,Não Assinado,Sem Conexão
1399,44538,Não se Aplica,Válido,Sem Conexão
2038,49172,Não se Aplica,Válido,OK
3040,56433,Não se Aplica,Não Assinado,Sem Conexão
1750,47281,Válido,Não se Aplica,OK
439,34380,Não se Aplica,Não Assinado,Sem Conexão
2517,50874,Não se Aplica,Não Assinado,Sem Conexão
2591,51551,NaN,NaN,Não informado


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
1549,46087,Fora do ACR,NaT,Nenhum
2006,48774,Fora do ACR,NaT,Nenhum
2544,50956,Fora do ACR,NaT,Nenhum
1463,44976,ACR,NaT,ACR
2249,49656,Fora do ACR,NaT,Nenhum
2185,49492,Fora do ACR,NaT,Nenhum
1766,47298,Fora do ACR,NaT,Nenhum
1020,38060,Fora do ACR,NaT,Nenhum
2343,49919,Fora do ACR,NaT,Nenhum
2236,49642,Fora do ACR,NaT,Nenhum


In [11]:
# Apenas mudança de nome
tabela_criterio = definicoes_monitoramento

In [12]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabela_criterio['criterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabela_criterio['dscriterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
607,35635,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
2738,52320,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1886,47459,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
149,31439,Em andamento,ACR,LI,OK,0,Usina em Obras em andamento
1119,38370,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
2501,50818,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
733,37086,Não Iniciada,ACL,LP,OK,1,Sem obras e PPA Ok
2439,50295,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1895,47468,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1424,44568,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"


In [13]:
slp = "Sem LP"
sim = "Sim"
tabela_criterio['criterio_novo'] = np.select(
    [
        (tabela_criterio.IdcSemPrevisao == sim),
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        (tabela_criterio.CondicaoAmbiental == "Sem LP"),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
449,34406,Em andamento,ACL,LO,OK,0,Usina em Obras em andamento
603,35631,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1689,47056,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1664,46890,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1903,47476,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"


In [14]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100


In [15]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento ']:
    if 'SigTipoGeracao' in calculo_previsao.columns:
        calculo_previsao.drop(['SigTipoGeracao'],inplace=True,axis=1)

In [208]:
tabela_criterio.shape

(2476, 39)

### criterios_da_previsao

In [232]:
criterios_da_previsao  = pd.merge(
                                    tabela_criterio,calculo_previsao,
                                    on="IdeUsinaOutorga",how='left'
                                    )
criterios_da_previsao = pd.merge(criterios_da_previsao,ug_rapeel,on='IdeUsinaOutorga',how='outer')

MemoryError: Unable to allocate 3.00 GiB for an array with shape (5, 80468484) and data type datetime64[ns]

In [230]:
criterios_da_previsao.shape

(9023505, 51)

In [227]:
################################ regra_antiga ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regra_antiga'] = pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regra_antiga'] = criterios_da_previsao[['Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regra_antiga'] = ( hoje +pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regra_antiga'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] >= 3) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[criterios_da_previsao['criterio'] >= 3 & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)


################################ regrapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regrapmo'] =  pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regrapmo'] = ( hoje + pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regrapmo'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8) ),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(3*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(4*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5) | (criterios_da_previsao['criterio'] == 7),'regrapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'regrapmo'] = pd.NA

KeyError: "['DatPrevistaComercial'] not in index"

In [ ]:
################################ dscjustificativapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dscjustificativapmo'] = "Analisar justificativa"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT") & ( criterios_da_previsao['Previsao_OC'] > criterios_da_previsao['DatPrevistaComercial']),'dscjustificativapmo'] = "Estágio Atual das Obras"
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT") & ( criterios_da_previsao['Previsao_OC'] < criterios_da_previsao['DatPrevistaComercial']),'dscjustificativapmo'] = "Estágio Atual das Obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dscjustificativapmo'] = "Usina liberada para testes"
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dscjustificativapmo'] = "Testes não concluídos"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Dat_OC_obrigacao']),'dscjustificativapmo'] = "Paralisação de Obras"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Dat_OC_obrigacao']),'dscjustificativapmo'] = "Paralisação de Obras"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Previsao_OC']),'dscjustificativapmo'] = "Estágio Atual das Obras"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Previsao_OC']),'dscjustificativapmo'] = "Estágio Atual das Obras" 
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'dscjustificativapmo'] = "Não celebrado contrato de uso do sistema de transmissão/distribuição"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4) | (criterios_da_previsao['criterio'] == 5),'dscjustificativapmo'] = "Situação da Licença de Instalação"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'dscjustificativapmo'] = "Sem Rapeel"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'dscjustificativapmo'] = "Sem Licenciamento Ambiental"


################################ criterionovopmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'criterionovopmo'] = 0.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'criterionovopmo'] = 0.2

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'criterionovopmo'] = 0.3

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 1),'criterionovopmo'] = 1

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 2),'criterionovopmo'] = 2

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 3),'criterionovopmo'] = 3

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 5),'criterionovopmo'] = 5

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 4),'criterionovopmo'] = 4

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 6),'criterionovopmo'] = 6

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 7),'criterionovopmo'] = 7

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 8),'criterionovopmo'] = 8

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 9),'criterionovopmo'] = 9

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 9),'dsccriterionovo'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterionovo'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterionovo'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterionovo'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 1),'dsccriterionovo'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 2),'dsccriterionovo'] = "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 3),'dsccriterionovo'] = "Usina sem obras, sem cust, sem PPA e com LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 4),'dsccriterionovo'] = "Usina sem obras, com cust, sem PPA e sem LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 5),'dsccriterionovo'] = "Usina sem obras, sem cust, sem PPA e sem LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 7),'dsccriterionovo'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 6),'dsccriterionovo'] = "Usina sem LP = Previsão OC sem previsão"
criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 8),'dsccriterionovo'] = "Usina obras paralisadas == Previsão OC maior entre data RAPEEL e data compromisso"

In [ ]:
################################ regranovapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regranovapmo'] = pd.NA

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].isna()),'regranovapmo'] = criterios_da_previsao['DatPrevistaComercial']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].notna()),'regranovapmo'] = criterios_da_previsao['Previsao_OC']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo']== 1) | (criterios_da_previsao['criterio_novo']== 2) ,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')

criterios_da_previsao.loc[((criterios_da_previsao['criterio_novo'] == 3) | (criterios_da_previsao['criterio_novo'] == 4) | (criterios_da_previsao['criterio_novo'] == 5) ),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 6,'regranovapmo'] = pd.NA

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(6*365,'D')
criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 7,'regranovapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 8,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)


################################ criterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'criterio_pmo_ts'] = 8

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'criterio_pmo_ts'] = 0.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'criterio_pmo_ts'] = 0.2

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'criterio_pmo_ts'] = 0.3

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'criterio_pmo_ts'] = 1.1

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 1.2

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 2.2

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'criterio_pmo_ts'] = 2.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'criterio_pmo_ts'] = 3

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'criterio_pmo_ts'] = 4

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'criterio_pmo_ts'] = 5

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'criterio_pmo_ts'] = 7

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'criterio_pmo_ts'] = 6

In [ ]:
################################ dsccriterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias == Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste == Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'dsccriterio_pmo_ts'] = "Usina sem PPA com LI sem CUST = Previsão OC maior entre handicap de 3 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI com CUST = Previsão OC maior entre handicap de 4 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI sem CUST = Previsão OC maior entre handicap de 5 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'dsccriterio_pmo_ts'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC maior entre handicap de 5 anos e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'dsccriterio_pmo_ts'] = "Usina sem LP = Previsão OC sem previsão"

In [ ]:
################################ criterio_pmo_ts_max_usina ################################
criterio_pmo_ts_max = criterios_da_previsao.groupby("IdeUsinaOutorga")[['criterio_pmo_ts']].max().rename(columns=
{'criterio_pmo_ts': 'criterio_pmo_ts_max_usina'})
criterios_da_previsao = criterios_da_previsao.merge(criterio_pmo_ts_max,on="IdeUsinaOutorga",how='left')


################################ dscjustificativaregra ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dscjustificativaregra'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['IdcObraParalisada'] == "Sim") & (criterios_da_previsao['IdcSemPrevisao'] != "Sim"),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.2),'dscjustificativaregra'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregra'] = "Estágio das obras da usina"


criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.1),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.2),'dscjustificativaregra'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 2.2),'dscjustificativaregra'] = "Licença de Instalação - LI e acesso a rede - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 3),'dscjustificativaregra'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregra'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregra'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregra'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregra'] = "Sem RAPEEL"


################################ dscjustificativaregranova ################################
criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 9),'dscjustificativaregranova'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 8),'dscjustificativaregranova'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 0.2),'dscjustificativaregranova'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregranova'] = "Estágio das obras da usina"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 1),'dscjustificativaregranova'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[criterios_da_previsao['criterionovopmo']==2,'dscjustificativaregranova'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[criterios_da_previsao['criterionovopmo']==3,'dscjustificativaregranova'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregranova'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregranova'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregranova'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregranova'] = "Sem RAPEEL"

In [ ]:
criterios_da_previsao.drop_duplicates(subset='dscjustificativaregranova')[['IdcSemPrevisao','criterio_pmo_ts_max_usina','criterionovopmo']]

,IdcSemPrevisao,criterio_pmo_ts_max_usina,criterionovopmo
0,Não,0.3,0.3
12,Não,2.2,2.0
115,Não,1.2,1.0
139,Não,5.0,5.0
189,Não,3.0,3.0
19739,Não,7.0,6.0
2236319,Não,2.1,8.0
3833107,Não,6.0,6.0


In [ ]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga","DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada"]
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols]

In [ ]:
# vInputCronograma tbm implementado,DATASREAL (LABEL="Idempreendimento" AS Idempreendimento)
cols = ["DatRealizacaoIII","DatRealizacaoV","DatRealizacaoXIV","DatRealizacaoXII","IdeUsinaOutorga","DatRealizacaoIV","DatRealizacaoIX","DatRealizacaoXI","DatRealizacaoX","DthEnvio","DatRealizacaoVI","DatRealizacaoII","CodCeg","DatRealizacaoXIII","DatRealizacaoVII"]
rename_cols = {'DatRealizacaoIII': 'IO_real','DatRealizacaoXIV': 'Conc_Trans_REAL','DatRealizacaoXII': 'Comiss_Real','DatRealizacaoIX': 'DesvRio_real','DatRealizacaoXI': 'Ench_Real','DthEnvio': 'Data_Envio','DatRealizacaoII': 'canteiroReal','DatRealizacaoXIII': 'IOTrans_Real','DatRealizacaoVII': 'ME_Real_conc_eol'}
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates()[cols].rename(columns=rename_cols)
vcronograma["classe"] = vcronograma.CodCeg.str.slice(3,5)
vcronograma["CC_real"] = pd.to_datetime(pd.NA)
vcronograma.loc[vcronograma.classe == "PH","CC_real"] = vcronograma.DatRealizacaoIV
vcronograma.loc[vcronograma.classe == "CV","CC_real"] = vcronograma.DatRealizacaoV
vcronograma["ME_real"] = vcronograma.DatRealizacaoX
vcronograma.loc[vcronograma.classe == "CV","ME_real"] = vcronograma.DatRealizacaoVI

In [ ]:
sas = pd.read_excel(r'C:\Users\Vinicius\OneDrive - unb.br\0Aneel\Previsor\WORK_definicoesmonitoramento.xlsx')

In [ ]:
teste = criterios_da_previsao.drop_duplicates(subset='IdeUsinaOutorga')

In [ ]:
merge =  pd.merge(teste,sas,on="IdeUsinaOutorga",how="inner",suffixes = ('_py','_sas'))

In [ ]:
mask_cusd_cust = (merge.NomSituacaoContratoCUSD_sas == merge.NomSituacaoContratoCUSD_py) & (merge.NomSitContratoCUST_py == merge.NomSitContratoCUST_sas)

mask_envio = (merge.DthEnvio_py == merge.DthEnvio_sas)

merge[(merge.CondicaoUso_py != merge.CondicaoUso_sas) & mask_cusd_cust & mask_envio][['CondicaoUso_py','CondicaoUso_sas','NomSituacaoContratoCUSD_sas','NomSituacaoContratoCUSD_py','NomSitContratoCUST_py','NomSitContratoCUST_sas','DthEnvio_py','DthEnvio_sas']]

,CondicaoUso_py,CondicaoUso_sas,NomSituacaoContratoCUSD_sas,NomSituacaoContratoCUSD_py,NomSitContratoCUST_py,NomSitContratoCUST_sas,DthEnvio_py,DthEnvio_sas
329,Sem Uso,OK,Válido,Válido,Não Assinado,Não Assinado,2022-12-05 17:36:44.277,2022-12-05 17:36:44.277
682,Sem Uso,OK,Válido,Válido,Não Assinado,Não Assinado,2022-12-04 11:32:40.297,2022-12-04 11:32:40.297


In [ ]:
mask_cusd_cust = (merge.DatValidadeLO_py == merge.DatValidadeLO_sas) & (merge.DatValidadeLI_py == merge.DatValidadeLI_sas)

merge[(merge.CondicaoAmbiental_py != merge.CondicaoAmbiental_sas) & mask_envio][['CondicaoAmbiental_py','CondicaoAmbiental_sas','DatValidadeLO_py','DatValidadeLO_sas']]

,CondicaoAmbiental_py,CondicaoAmbiental_sas,DatValidadeLO_py,DatValidadeLO_sas


In [ ]:
merge[(merge.CondicaoAmbiental_py != merge.CondicaoAmbiental_sas) & mask_cusd_cust & mask_envio][['CondicaoAmbiental_py','CondicaoAmbiental_sas']]

,CondicaoAmbiental_py,CondicaoAmbiental_sas


In [ ]:
merge[merge.PPA_sas != merge.PPA_py]

,IdeUsinaOutorga,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra_x,IdcObraParalisada_py,IdcUsinaMonitorada_py,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado_x,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra_py,IdcSemPrevisao,DatComissionamentoUGRealizado,DatMonitoramento_x,SigTipoGeracao_py,DscComercializacaoEnergia_py,DatInicioObraRealizado,DatMontagemRealizado,DthEnvio_py,DatConclusaoACL_py,DatValidadeLI_py,DatValidadeLP_py,DatValidadeLO_py,NomSitContratoCCD_py,NomSitContratoCUST_py,NomSituacaoContratoCUSD_py,NomSitContratoCCT_py,NumOperacaoUg,DatPrevistaComercial,ValidadeAmbiental_py,CondicaoAmbiental_py,CondicaoConexao_py,CondicaoUso_py,PPA_py,criterio,dscriterio,criterio_novo,NomUsina_py,NumUgUsina,Previsao_OC,Dat_OC_obrigacao,DatMonitoramento_y,FASE,Ind_crono_norm,flagOPTeste30dias,DatPrevisaoIniciobra_y,...,SituacaoACL,PctEnergiaContratoAssinadoACL,DatConclusaoACL_sas,IdcTipoInstalacao,IdcNecessidade,NomSituacaoParAcesso,DatEmissaoParAcesso,NomSitContratoCCD_sas,DatAssinaturaContratoCCD,DatValidadeContratoCCD,NomSitContratoCCT_sas,DatAssinaturaContratoCCT,DatValidadeContratoCCT,NomSituacaoContratoCUSD_sas,DatAssinaturaContratoCUSD,DatValidadeContratoCUSD,NomSitContratoCUST_sas,DatAssinaturaContratoCUST,DatValidadeContratoCUST,DatSolicitacaoLP,DatPrevistaEmissaoLP,DatEmissaoLP,DatValidadeLP_sas,NomSituacaoLP,DatSolicitacaoLI,DatPrevistaEmissaoLI,DatEmissaoLI,DatValidadeLI_sas,NomSituacaoLI,DatSolicitacaoLO,DatPrevistaEmissaoLO,DatEmissaoLO,DatValidadeLO_sas,NomSituacaoLO,IdcTipoEntrave,DscSolucao,DscObservacaoAcesso,DscObservacaomercado,DscObservacaoAmbiental,ValidadeAmbiental_sas,CondicaoAmbiental_sas,CondicaoConexao_sas,CondicaoUso_sas,PPA_sas,DscComercializacaoEnergia_sas,DscSistema,DatPrevistaComercialRapeelMX,DscJustificativaPrevisao,DscObservacaoUG,DatEmissaoAto
